In [1]:
from PIL import Image
import pandas as pd
import torch
from transformers import CLIPProcessor, CLIPModel

# Load the dataset
dataset_path = 'reverse_image_search.csv'  # Replace with your dataset path
df = pd.read_csv(dataset_path)

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")




In [2]:
# Initialize a list to store tensors
embeddings = []

for index, row in df.iterrows():
    image_path = row['path']  # Assuming the path is in a column named 'path'
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB
    inputs = processor(images=image, return_tensors="pt")
    image_features = model.get_image_features(**inputs)
    # Ensure the tensor is detached from the computational graph before converting
    embeddings.append(image_features.squeeze(0).detach().numpy().tolist())

# Concatenate all feature vectors into a single tensor
#image_features_tensor = torch.stack(embeddings)

# image_features_tensor now contains the feature vectors for all images in your dataset

In [52]:
# image_features

In [27]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

# Milvus parameters
HOST = '127.0.0.1'
PORT = '19530'
TOPK = 13

In [28]:
connections.connect(host=HOST, port=PORT)
collection_name = 'tranformers_clip'
dim = 512  # Dimension of the embeddings
METRIC_TYPE = 'L2'  # You can choose 'L2', 'IP', etc., based on your requirement
INDEX_TYPE = 'IVF_FLAT'  # Index type

In [29]:
utility.list_collections()

['text_image_search_blip',
 'image_search_collection',
 'text_image_search',
 'tranformers_clip',
 'reverse_image_search']

In [30]:
def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
        FieldSchema(name='path', dtype=DataType.VARCHAR, description='path to image', max_length=500, 
                    is_primary=True, auto_id=False),
        FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, description='image embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='reverse image search')
    collection = Collection(name=collection_name, schema=schema)

    index_params = {
        'metric_type': METRIC_TYPE,
        'index_type': INDEX_TYPE,
        'params': {"nlist": 512}
    }
    collection.create_index(field_name='embedding', index_params=index_params)
    return collection

In [31]:
collection = create_milvus_collection(collection_name, dim)

In [32]:
paths = df['path'].tolist()


In [51]:
# embeddings[7] #how an embedding looks like

In [34]:
entities = [[path for path in paths],
            [embedding for embedding in embeddings]]
#preparing for insertion to milvus

In [35]:
mr = collection.insert(entities)


In [36]:
# Query processing for search


In [37]:
collection = Collection(collection_name)      # Get an existing collection.
collection.load()

In [38]:
search_params = {
    "metric_type": "L2", 
    "offset": 0, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}

In [47]:


query_image_path = 'brain_coral.jpg'  
query_image = Image.open(query_image_path).convert('RGB')  
query_inputs = processor(images=query_image, return_tensors="pt")
query_image_features = model.get_image_features(**query_inputs)
embedding = query_image_features.squeeze(0).detach().numpy().tolist()


# Concatenate all feature vectors into a single tensor
#image_features_tensor = torch.stack(embeddings)

# image_features_tensor now contains the feature vectors for all images in your dataset


In [48]:
results = collection.search(
    data=[embedding], 
    anns_field="embedding", 
    # the sum of `offset` in `param` and `limit` 
    # should be less than 16384.
    param=search_params,
    limit=10,
    expr=None,
)


In [49]:
results[0].ids


['./train/brain_coral/n01917289_2484.JPEG', './train/brain_coral/n01917289_1079.JPEG', './train/brain_coral/n01917289_4021.JPEG', './train/brain_coral/n01917289_765.JPEG', './train/brain_coral/n01917289_1082.JPEG', './train/brain_coral/n01917289_4069.JPEG', './train/brain_coral/n01917289_1783.JPEG', './train/brain_coral/n01917289_1538.JPEG', './train/brain_coral/n01917289_4317.JPEG', './train/brain_coral/n01917289_1022.JPEG']

In [50]:
results[0].distances

[23.210317611694336, 27.270641326904297, 34.20412826538086, 34.476402282714844, 42.19493103027344, 43.957298278808594, 44.653133392333984, 54.235069274902344, 62.594215393066406, 73.49400329589844]